<a href="https://colab.research.google.com/github/tousif47/Mini-LLM/blob/main/Scratch_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import re

from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

print("Done")

Done


In [24]:
data_text = open("/content/drive/MyDrive/Practice Playing with code/health.txt", "r")

print(data_text.read(100), "\n\nNumber of characters in text file :", len(data_text.read()))

Being healthy and fit in simple terms means taking good care of the body. We should remember that a  

Number of characters in text file : 7030


In [25]:
# Preprocessing the text data
# Basic text preprocessing because a simple dataset with minimal noise has been selected to simplify the code
# To maintain uniformity all words has been converted to lower case and any words with length less than 3 has been removed
# Punctuations are also removed

def text_preprocess(text):

    # Lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)

    # Remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString)
    long_words = []

    # Remove short words
    for i in newString.split():
        if len(i) >= 3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

# Calling the preprocessor
data_text = open("/content/drive/MyDrive/Practice Playing with code/health.txt", "r")
data_preprocessed = text_preprocess(data_text.read())

print(data_preprocessed[:100])

being healthy and fit simple terms means taking good care the body should remember that healthy mind


In [26]:
# Creating sequences
# Taking 40 characters as context, model will try to predict the next character

def sequencer(text):
    length = 40
    sequences = list()

    for i in range(length, len(text)):

        # Select sequence of tokens
        seq = text[i-length:i+1]

        # Store
        sequences.append(seq)

    print("Total Sequences: %d" % len(sequences))
    return sequences

# Calling the sequencer
sequences = sequencer(data_preprocessed)
print("\n", sequences[0], "\n", sequences[1], "\n", sequences[2], "\n", sequences[3], "\n", sequences[4])

Total Sequences: 6322

 being healthy and fit simple terms means  
 eing healthy and fit simple terms means t 
 ing healthy and fit simple terms means ta 
 ng healthy and fit simple terms means tak 
 g healthy and fit simple terms means taki


In [27]:
# Encoding the sequences

chars = sorted(list(set(data_preprocessed)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encoder(seq):

    sequences = list()

    for line in seq:

        # Integer encode line
        encoded = [mapping[char] for char in line]

        # Store
        sequences.append(encoded)

    return sequences

# Calling the encoder
encoded_seq = encoder(sequences)
print(encoded_seq[0], "\n", encoded_seq[1], "\n", encoded_seq[2], "\n", encoded_seq[3], "\n", encoded_seq[4])

[2, 5, 9, 14, 7, 0, 8, 5, 1, 12, 20, 8, 25, 0, 1, 14, 4, 0, 6, 9, 20, 0, 19, 9, 13, 16, 12, 5, 0, 20, 5, 18, 13, 19, 0, 13, 5, 1, 14, 19, 0] 
 [5, 9, 14, 7, 0, 8, 5, 1, 12, 20, 8, 25, 0, 1, 14, 4, 0, 6, 9, 20, 0, 19, 9, 13, 16, 12, 5, 0, 20, 5, 18, 13, 19, 0, 13, 5, 1, 14, 19, 0, 20] 
 [9, 14, 7, 0, 8, 5, 1, 12, 20, 8, 25, 0, 1, 14, 4, 0, 6, 9, 20, 0, 19, 9, 13, 16, 12, 5, 0, 20, 5, 18, 13, 19, 0, 13, 5, 1, 14, 19, 0, 20, 1] 
 [14, 7, 0, 8, 5, 1, 12, 20, 8, 25, 0, 1, 14, 4, 0, 6, 9, 20, 0, 19, 9, 13, 16, 12, 5, 0, 20, 5, 18, 13, 19, 0, 13, 5, 1, 14, 19, 0, 20, 1, 11] 
 [7, 0, 8, 5, 1, 12, 20, 8, 25, 0, 1, 14, 4, 0, 6, 9, 20, 0, 19, 9, 13, 16, 12, 5, 0, 20, 5, 18, 13, 19, 0, 13, 5, 1, 14, 19, 0, 20, 1, 11, 9]


In [29]:
# Creating training and testing split from the dataset

# Vocabulary size
vocabulary = len(mapping)
encoded_seq = np.array(encoded_seq)

X, y = encoded_seq[:,:-1], encoded_seq[:,-1]

# One hot encode y
y = to_categorical(y, num_classes = vocabulary)

# Training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 47)

print("Train set:", X_train.shape, "Test set:", X_test.shape)

Train set: (5057, 40) Test set: (1265, 40)


In [30]:
# Building the LLM
# Embedding layer of Keras is used to creat a 50 dimension embedding for each character
# GRU layer is used as base model with 150 timestamps
# A Dense layer is used with a softmax activation function for prediction

# Defining
model = Sequential()
model.add(Embedding(vocabulary, 50, input_length = 40, trainable = True))
model.add(GRU(150, recurrent_dropout = 0.1, dropout = 0.1))
model.add(Dense(vocabulary, activation = "softmax"))

print(model.summary())

# Compiling
model.compile(loss = "categorical_crossentropy", metrics = ["acc"], optimizer="adam")

# Fitting
model.fit(X_train, y_train, epochs = 100, verbose = 2, validation_data = (X_test, y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            1350      
                                                                 
 gru (GRU)                   (None, 150)               90900     
                                                                 
 dense (Dense)               (None, 27)                4077      
                                                                 
Total params: 96,327
Trainable params: 96,327
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
159/159 - 31s - loss: 2.8417 - acc: 0.1859 - val_loss: 2.5719 - val_acc: 0.2482 - 31s/epoch - 195ms/step
Epoch 2/100
159/159 - 25s - loss: 2.3649 - acc: 0.2994 - val_loss: 2.2952 - val_acc: 0.2941 - 25s/epoch - 160ms/step
Epoch 3/100
159/159 - 23s - loss: 2.1723 - acc: 0.3548 - val_loss: 2.1964 - v

In [35]:
# Generating a sequence of characters with the LLM
# The LLM will attempt to predict the next 100 characters after taking bla bla

def generator(model, mapping, seq_length, seed_text, n_chars):

	input_text = seed_text

	# Generate a fixed number of characters
	for _ in range(n_chars):

		# Encode the characters as integers
		encoded = [mapping[char] for char in input_text]

		# Truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen = seq_length, truncating = "pre")

		# Predict character
		yhat = np.argmax(model.predict(encoded), axis = -1)

		# Reverse map integer to character
		out_char = ""

		for char, index in mapping.items():

			if index == yhat:
				out_char = char
				break

		# Append
		input_text += char

	return input_text

# Calling the generator
input = "LLM is making new sentences "

print(len(input))
print(generator(model, mapping, 40, input.lower(), 100))

28
1/1 [==============================] - 0s 53ms/step
llm is making new sentences that defing not only look and feel good but comporumetain and taking part healthy and fit lifestyle 
